# Pre-processing Nevada Allocation data for WaDEQA upload.
Date Updated: 08/03/2020
Purpose:  To pre-process the Nevada data into one master file for simple DataFrame creation and extraction

### Notes:
To incldue owners, made a temporary Permit_Owners_5temp.csv file by removing previous onwers from list.  Plan on left-join to POD AllApps_2.csv by 'app' field.


https://ndwr.maps.arcgis.com/home/item.html?id=0d050f7b79724404b80bf29589f67363
https://arcgis.shpo.nv.gov/arcgis/rest/services/Water_Resources_Public_Data/WaterRights_POD_POU/FeatureServer
https://arcgis.shpo.nv.gov/arcgis/rest/services/Water_Resources_Public_Data/SE_HydrographicBasins/FeatureServer
http://water.nv.gov/CodeDefinitions.aspx

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [2]:
#Working Directory
workingDir = "G:/Shared drives/WaDE Data/Nevada/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Input Data

In [3]:
# POD sites Data
PoDAAInput = "POD AllApps_2_input.zip"
dfPoD = pd.read_csv(PoDAAInput).replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPoD:
    dfPoD['WaDEUUID'] = "nvD" + dfPoD.index.astype(str)
    dfPoD.to_csv('POD AllApps_2_input.zip', compression=dict(method='zip', archive_name='POD AllApps_2_input.csv'), index=False)
    
dfPoD['in_PODorPOUSite'] = "POD"
dfPoD['in_SiteNativeID'] = "POD" + dfPoD.index.astype(str) # creating custom site Native iD for POD

print(len(dfPoD))
dfPoD.head(1)

C:\Users\rjame\AppData\Local\Temp\ipykernel_16940\1632620707.py:3: DtypeWarning: Columns (5,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  dfPoD = pd.read_csv(PoDAAInput).replace(np.nan, "")


102951


,WaDEUUID,ReasonRemoved,IncompleteField,OBJECTID,app,cert,app_status,mou,site_name,permit_info,basin,county,source,duty_balance,div_balance,prior_dt,pod_twn,pod_rng,pod_sec,pod_q,pod_qq,latitude,longitude,poly_id,sup_group,x,y,in_PODorPOUSite,in_SiteNativeID
0,nvD0,,,1,15809,,DEN,IRR,,http://water.nv.gov/permitinformation.aspx?app...,1,HU,STR,746.67,4.00000,9/23/1954 0:00,47N,31E,5.0,LT03,,41.98833,-118.52723,,,-118.52723,41.98833,POD,POD0


In [4]:
# Input File - PoU Shapefile Data
# export dataframe as zipped csv
PoUAAInput = 'shapefile/NVwr_POU.zip'
dfPoU = gpd.read_file(PoUAAInput).replace(np.nan, "")
dfPoU = dfPoU.drop(['geometry'], axis=1)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPoU:
    dfPoU['WaDEUUID'] = "nvU" + dfPoU.index.astype(str)
    dfPoU.to_csv('NVwr_POU.zip', compression=dict(method='zip', archive_name='NVwr_POU.csv'), index=False)
    
dfPoU['in_PODorPOUSite'] = "POU"
dfPoU['in_SiteNativeID'] = "POU" + dfPoU['wadeSiteID']

print(len(dfPoU))
dfPoU.head(1)

29161


,app,cert,app_status,mou,site_name,permit_inf,basin,county,source,duty_balan,div_balanc,prior_dt,pou_acre_t,poly_id,wadeSiteID,Latitude,Longitude,GlobalID,Shape_Leng,Shape_Area,WaDEUUID,in_PODorPOUSite,in_SiteNativeID
0,V11569,,VST,COM,036 N39 E52 15DBCC2,http://water.nv.gov/permitinformation.aspx?app...,036,EL,UG,0.00000,0.20000,1929-01-01,0.00000,69906,wadeID1,41.27617,-116.11752,{A4691461-1E67-43AD-9D4D-D525764C8A22},2309.90802,194139.97513,nvU0,POU,POUwadeID1


In [5]:
# Owner Data
OwnTemp = "Permit_Owners_5temp.zip"
dfown = pd.read_csv(OwnTemp).replace(np.nan, "")

# With owner sort and merge columns by 'app' field.
dfown = dfown.groupby('app', sort=False).agg(lambda x: ', '.join([str(elem) for elem in (list(set(x)))]))
dfown = dfown.drop_duplicates().reset_index()
print(len(dfown))
dfown.head(1)

C:\Users\rjame\AppData\Local\Temp\ipykernel_16940\2408382377.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dfown = pd.read_csv(OwnTemp).replace(np.nan, "")


104112


,app,OBJECTID,owner_name,owner_type,owner_pct,owner_duty,owner_div_rate,owner_acre,owner_chg_app,county,apn,owner_remark
0,1,1,"PRATT, A.C.",B,0.0,0.0,0.0,0.0,,,,


# POD Data

In [6]:
# Merge owner data with POD data 
dfPoD = pd.merge(dfPoD, dfown, left_on='app', right_on='app', how='left')
dfPoD.head()

,WaDEUUID,ReasonRemoved,IncompleteField,OBJECTID_x,app,cert,app_status,mou,site_name,permit_info,basin,county_x,source,duty_balance,div_balance,prior_dt,pod_twn,pod_rng,pod_sec,pod_q,pod_qq,latitude,longitude,poly_id,sup_group,x,y,in_PODorPOUSite,in_SiteNativeID,OBJECTID_y,owner_name,owner_type,owner_pct,owner_duty,owner_div_rate,owner_acre,owner_chg_app,county_y,apn,owner_remark
0,nvD0,,,1,15809,,DEN,IRR,,http://water.nv.gov/permitinformation.aspx?app...,1,HU,STR,746.67,4.00000,9/23/1954 0:00,47N,31E,5.0,LT03,,41.98833,-118.52723,,,-118.52723,41.98833,POD,POD0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nvD1,,,2,18001,5536,CER,IRR,001 N47 E30 04B 2,http://water.nv.gov/permitinformation.aspx?app...,1,HU,UG,32.58,0.04500,5/14/1959 0:00,47N,30E,4.0,TR38,,41.98868,-118.63691,,,-118.63691,41.98868,POD,POD1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nvD10,,,11,25883,,CAN,IRR,,http://water.nv.gov/permitinformation.aspx?app...,1,HU,UG,44.96,0.25000,12/7/1970 0:00,47N,30E,19.0,TR40,,41.93528,-118.66167,,,-118.66167,41.93528,POD,POD2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nvD100,,,101,78603,,RFP,MUN,021 N30 E19 10DBDC1,http://water.nv.gov/permitinformation.aspx?app...,21,WA,UG,182.28,0.70000,1/1/1924 0:00,30N,19E,10.0,SE,NW,40.48571,-119.82966,,,-119.82966,40.48571,POD,POD3,159307,"BRIGHT HOLLAND CORPORATION, A NEVADA CORPORATION",B,0.0,0.0,0.0,0.0,,,,
4,nvD1000,,,1001,46388,,DEN,IRD,,http://water.nv.gov/permitinformation.aspx?app...,9,WA,UG,1240.0,5.40000,11/23/1982 0:00,41N,20E,19.0,SE,NE,41.45528,-119.77583,,,-119.77583,41.45528,POD,POD4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfPoD['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "NVwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "NVwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "NVwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = ""
df['in_WaterSourceNativeID'] = " "# auto fill in below
df['in_WaterSourceTypeCV'] = dfPoD['source']

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = "Digitized"
df['in_County'] = dfPoD['county_x']
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfPoD['y']
df['in_Longitude'] = dfPoD['x']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = dfPoD['in_PODorPOUSite']
df['in_SiteName'] = dfPoD['site_name']
df['in_SiteNativeID'] = dfPoD['in_SiteNativeID']
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfPoD['source']
df['in_StateCV'] = "NV"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = ""
df['in_AllocationLegalStatusCV'] = dfPoD['app_status']
df['in_AllocationNativeID'] =  dfPoD['app'].replace("", 0).fillna(0).astype(str).str.lower().str.strip()
df['in_AllocationOwner'] = dfPoD['owner_name']
df['in_AllocationPriorityDate'] = dfPoD['prior_dt']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfPoD['duty_balance']
df['in_BeneficialUseCategory'] = dfPoD['mou']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfPoD['permit_info']

outdfPoD = df.copy()
outdfPoD = outdfPoD.drop_duplicates().reset_index(drop=True)
print(len(outdfPoD))
outdfPoD.head()

102951


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,nvD0,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,STR,,Digitized,HU,4326,,,,41.98833,-118.52723,,,POD,,POD0,,STR,NV,,,,,,,,,,,DEN,15809,NaN,9/23/1954 0:00,,,,,746.67,IRR,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
1,nvD1,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,HU,4326,,,,41.98868,-118.63691,,,POD,001 N47 E30 04B 2,POD1,,UG,NV,,,,,,,,,,,CER,18001,NaN,5/14/1959 0:00,,,,,32.58,IRR,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
2,nvD10,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,HU,4326,,,,41.93528,-118.66167,,,POD,,POD2,,UG,NV,,,,,,,,,,,CAN,25883,NaN,12/7/1970 0:00,,,,,44.96,IRR,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
3,nvD100,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,WA,4326,,,,40.48571,-119.82966,,,POD,021 N30 E19 10DBDC1,POD3,,UG,NV,,,,,,,,,,,RFP,78603,"BRIGHT HOLLAND CORPORATION, A NEVADA CORPORATION",1/1/1924 0:00,,,,,182.28,MUN,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
4,nvD1000,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,WA,4326,,,,41.45528,-119.77583,,,POD,,POD4,,UG,NV,,,,,,,,,,,DEN,46388,NaN,11/23/1982 0:00,,,,,1240.0,IRD,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...


# POU Data

In [8]:
# Merge owner data with POU data 
dfPoU = pd.merge(dfPoU, dfown, left_on='app', right_on='app', how='left')
dfPoU.head()

,app,cert,app_status,mou,site_name,permit_inf,basin,county_x,source,duty_balan,div_balanc,prior_dt,pou_acre_t,poly_id,wadeSiteID,Latitude,Longitude,GlobalID,Shape_Leng,Shape_Area,WaDEUUID,in_PODorPOUSite,in_SiteNativeID,OBJECTID,owner_name,owner_type,owner_pct,owner_duty,owner_div_rate,owner_acre,owner_chg_app,county_y,apn,owner_remark
0,V11569,,VST,COM,036 N39 E52 15DBCC2,http://water.nv.gov/permitinformation.aspx?app...,036,EL,UG,0.00000,0.20000,1929-01-01,0.00000,69906,wadeID1,41.27617,-116.11752,{A4691461-1E67-43AD-9D4D-D525764C8A22},2309.90802,194139.97513,nvU0,POU,POUwadeID1,195256,"YOUNG, THOMAS A.",B,0.0,0.0,0.0,0.0,,,,
1,V11519,,VST,IRR,033A N43 E37 34BBBD1,http://water.nv.gov/permitinformation.aspx?app...,033A,HU,UG,0.00000,0.00000,1923-01-01,0.00000,70212,wadeID2,41.56777,-117.79997,{5AA23025-1327-41BB-8083-FB1754B95C25},4004.29090,640308.96333,nvU1,POU,POUwadeID2,195206,"ST. CLAIR, RODNEY",B,0.0,0.0,0.0,0.0,,,,
2,V11518,,VST,IRR,033A N43 E37 29DDCB1,http://water.nv.gov/permitinformation.aspx?app...,033A,HU,UG,0.00000,0.00000,1919-01-01,0.00000,70211,wadeID3,41.57357,-117.83477,{8CFB80D4-B00D-4A2F-9F1A-65A342B5141C},2851.62099,499295.62322,nvU2,POU,POUwadeID3,195205,"ST. CLAIR, RODNEY",B,0.0,0.0,0.0,0.0,,,,
3,V11430,,VST,COM,087 N19 E19 17ADDC1,http://water.nv.gov/permitinformation.aspx?app...,087,WA,UG,4.00000,0.00000,1938-12-31,0.00000,70165,wadeID4,39.51233,-119.86583,{92C00799-B64A-4FD2-9BE0-1115B491B5CD},284.53904,4134.62605,nvU3,POU,POUwadeID4,195117,"BAJWA PROPERTIES, LLC",B,0.0,0.0,0.0,0.0,,,,
4,V11425,,VST,COM,001 N47 E30 04B 6,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00000,0.00000,2018-06-19,0.00000,70162,wadeID5,41.98953,-118.63680,{A99F140D-A917-4C11-8B18-A1A8B18E7219},572.32726,11863.85168,nvU4,POU,POUwadeID5,195109,"CLIFTON, CLAY",B,0.0,0.0,0.0,0.0,,,,


In [9]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfPoU['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "NVwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "NVwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "NVwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = ""
df['in_WaterSourceNativeID'] = " "# auto fill in below
df['in_WaterSourceTypeCV'] = dfPoU['source']

# Site Info
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = "Digitized"
df['in_County'] = dfPoU['county_x']
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfPoU['Latitude']
df['in_Longitude'] = dfPoU['Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = dfPoU['in_PODorPOUSite']
df['in_SiteName'] = dfPoU['site_name']
df['in_SiteNativeID'] = dfPoU['in_SiteNativeID']
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfPoU['source']
df['in_StateCV'] = "NV"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = ""
df['in_AllocationLegalStatusCV'] = dfPoU['app_status']
df['in_AllocationNativeID'] =  dfPoU['app'].replace("", 0).fillna(0).astype(str).str.lower().str.strip()
df['in_AllocationOwner'] = dfPoU['owner_name']
df['in_AllocationPriorityDate'] = dfPoU['prior_dt']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = ""
df['in_AllocationTimeframeStart'] = ""
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfPoU['duty_balan']
df['in_BeneficialUseCategory'] = dfPoU['mou']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfPoU['permit_inf']

outdfPoU = df.copy()
outdfPoU = outdfPoU.drop_duplicates().reset_index(drop=True)
print(len(outdfPoU))
outdfPoU.head()

29161


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,nvU0,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,EL,4326,,,,41.27617,-116.11752,,,POU,036 N39 E52 15DBCC2,POUwadeID1,,UG,NV,,,,,,,,,,,VST,v11569,"YOUNG, THOMAS A.",1929-01-01,,,,,0.00000,COM,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
1,nvU1,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,HU,4326,,,,41.56777,-117.79997,,,POU,033A N43 E37 34BBBD1,POUwadeID2,,UG,NV,,,,,,,,,,,VST,v11519,"ST. CLAIR, RODNEY",1923-01-01,,,,,0.00000,IRR,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
2,nvU2,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,HU,4326,,,,41.57357,-117.83477,,,POU,033A N43 E37 29DDCB1,POUwadeID3,,UG,NV,,,,,,,,,,,VST,v11518,"ST. CLAIR, RODNEY",1919-01-01,,,,,0.00000,IRR,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
3,nvU3,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,WA,4326,,,,39.51233,-119.86583,,,POU,087 N19 E19 17ADDC1,POUwadeID4,,UG,NV,,,,,,,,,,,VST,v11430,"BAJWA PROPERTIES, LLC",1938-12-31,,,,,4.00000,COM,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
4,nvU4,NVwr_M1,NVwr_V1,NVwr_O1,,,,,,UG,,Digitized,HU,4326,,,,41.98953,-118.63680,,,POU,001 N47 E30 04B 6,POUwadeID5,,UG,NV,,,,,,,,,,,VST,v11425,"CLIFTON, CLAY",2018-06-19,,,,,0.00000,COM,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...


## Concatenate POD & POU

In [10]:
# Concatenate dataframes
frames = [outdfPoD, outdfPoU]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

132112


In [11]:
# For creating County
CountyDict = {
    "HU" : "Humboldt",
    "CC" : "Carson City",
    "CH" : "Churchill",
    "CL" : "Clark",
    "DO" : "Douglas",
    "EL" : "Elko",
    "ES" : "Esmerelda",
    "EU" : "Eureka",
    "LA" : "Lander",
    "LI" : "Lincoln",
    "LY": "Lyon",
    "MI": "Mineral",
    "NY": "Nye",
    "PE": "Pershing",
    "ST": "Storey",
     "": "Unknown",
    "WA": "Washoe",
    "WP": "White Pine",
    "UK": "Unknown"}
def assignCounty(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ""
    else:
        String1 = colrowValue.strip()
        try:
            outList = CountyDict[String1]
        except:
            outList = ""
    return outList

outdf['in_County'] = outdf.apply(lambda row: assignCounty(row['in_County']), axis=1)
outdf['in_County'].unique() 

array(['Humboldt', 'Washoe', 'Elko', 'Lyon', 'Nye', 'Clark', 'Douglas',
       'Lincoln', 'Pershing', 'Lander', 'White Pine', 'Eureka',
       'Esmerelda', 'Churchill', 'Mineral', 'Storey', 'Carson City',
       'Unknown', ''], dtype=object)

In [12]:
# For creating AllocationLegalStatusCV
LegalDict = {
"ABN" : "Abandoned",
"ABR" : "Abrogated",
"APP" : "Application",
"CAN" : "Canceled",
"CER" : "Certificate",
"CUR" : "Curtailed",
"DEC" : "Decreed",
"DEN": "Denied",
"EXP": "Expired",
"FOR": "Forfeited",
"PER": "Permit",
"REJ": "Rejected",
"REL": "Relinquished",
"RES": "Reserved",
"RFA": "Ready For Action",
"RFP": "Ready for Action (Protested)",
"RLP": "Relinquish a Portion",
"RSC": "Rescinded",
"RVK": "Revoked",
"RVP": "Revocable Permit",
"SUP": "Supersceded",
"SUS": "Suspended",
"VST": "Vested Rights",
"WDR": "Withdrawn"}
def assignAllocationLegalStatusCV(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue) == True :
        outList = ""
    else:
        String1 = colrowValue.strip()
        try:
            outList = LegalDict[String1]
        except:
            outList = ""
    return outList
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: assignAllocationLegalStatusCV(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['Denied', 'Certificate', 'Canceled',
       'Ready for Action (Protested)', 'Permit', 'Abrogated', 'Withdrawn',
       'Expired', 'Revocable Permit', 'Decreed', 'Relinquish a Portion',
       'Ready For Action', 'Vested Rights', 'Forfeited', 'Reserved',
       'Abandoned', 'Supersceded', 'Application', 'Rejected',
       'Relinquished', 'Revoked', 'Rescinded', 'Suspended', 'Curtailed'],
      dtype=object)

In [13]:
# For creating BeneficialUse
BeneficialUseDict = {
"COM" : "Commercial",
"CON" : "Construction",
"DEC" : "As Decreed",
"DOM" : "Domestic",
"DWR" : "Dewatering",
"ENV" : "Environmental",
"EVP" : "Evaporation",
"IND": "Industrial",
"IRC": "Irrigation-Carey Act",
"IRD": "Irrigation-DLE",
"IRR": "Irrigation",
"MM": "Mining and Milling",
"MMD": "Mining Milling and Dewatering",
"MUN": "Municipal",
"OTH": "Other",
"PWR": "Power",
"QM": "Quasi-Municipal",
"REC": "Recreational",
"STK": "Stockwatering",
"STO": "Storage",
"UKN": "Unknown",
"WLD": "Wildlife"}
def assignBeneficialUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = BeneficialUseDict[String1]
        except:
            outList = ""

    return outList
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: assignBeneficialUse(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array(['Irrigation', 'Municipal', 'Irrigation-DLE', 'Other', 'Commercial',
       'Quasi-Municipal', 'Mining and Milling', 'Industrial',
       'Stockwatering', 'Mining Milling and Dewatering', 'Domestic',
       'Wildlife', 'Environmental', 'As Decreed', 'Construction',
       'Dewatering', 'Recreational', 'Power', 'Storage',
       'Irrigation-Carey Act', 'Evaporation', 'Unknown'], dtype=object)

In [14]:
# For creating WaterSourceTypeCV
UnknownWSCVDict = {
"EFF" : "Reuse",
"GEO" : "Groundwater",
"LAK" : "Surface Water",
"OGW" : "Groundwater",
"OSW" : "Surface Water",
"RES" : "Reservoir",
"SPR" : "Surface Water",
"STO" : "Storage",
"STR" : "Surface Water",
"UG" : "Groundwater",
"UKN" : "Unknown"}
def assignWaterSourceTypeCV(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = ""
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = UnknownWSCVDict[String1]
        except:
            outList = ""
    return outList

outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: assignWaterSourceTypeCV(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['Surface Water', 'Groundwater', 'Reuse', 'Reservoir', 'Storage',
       'Unknown'], dtype=object)

In [15]:
# For creating SiteTypeCV
UnknownSTCVDict = {
    "EFF":"Effluent",
    "GEO":"Geothermal",
    "LAK":"lake",
    "OGW":"Other Ground Water",
    "OSW":"Other Surface Water",
    "RES":"Reservoir",
    "SPR":"Spring",
    "STO":"Storage",
    "STR":"stream",
    "UG":"Underground",
    "UKN":"Unknown"}
def assignSiteTypeCV(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ""
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = UnknownSTCVDict[String1]
        except:
            outList = ""
    return outList

outdf['in_SiteTypeCV'] = outdf.apply(lambda row: assignSiteTypeCV(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['stream', 'Underground', 'Geothermal', 'Spring',
       'Other Surface Water', 'Other Ground Water', 'Effluent', 'lake',
       'Reservoir', 'Storage', 'Unknown'], dtype=object)

## Clean Data & WaDE Custom Elements

In [16]:
# Clean owner name up
def removeSpecialCharsFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().replace("  ", " ").strip()
    return Val

In [17]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['', 'Bright Holland Corporation A Nevada Corporation',
       'Sherlock Donald E Sherlock Arletta G', ..., 'Sloane Violet H',
       'Ehrhart Thomas A Jr',
       'Washburn Family Trust Dated February 19 2010'], dtype=object)

In [18]:
# some POD source data has a few names that contain a ',' in them, but should still be okay
outdf['in_SiteName'] = outdf.apply(lambda row: removeSpecialCharsFunc(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['', '001 N47 E30 04B  2', '021 N30 E19 10Dbdc1', ...,
       '212 S19 E60 26Dcdc1', '176 N24 E58 16Bbcc1',
       '107 N11 E24 20Cbcb2'], dtype=object)

In [19]:
# Ensure Empty String

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [20]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['Surface Water', 'Groundwater', 'Reuse', 'Reservoir', 'Storage',
       'Unknown'], dtype=object)

In [21]:
outdf['in_County'] = outdf.apply(lambda row: ensureEmptyString(row['in_County']), axis=1)
outdf['in_County'].unique()

array(['Humboldt', 'Washoe', 'Elko', 'Lyon', 'Nye', 'Clark', 'Douglas',
       'Lincoln', 'Pershing', 'Lander', 'White Pine', 'Eureka',
       'Esmerelda', 'Churchill', 'Mineral', 'Storey', 'Carson City',
       'Unknown', ''], dtype=object)

In [22]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['', '001 N47 E30 04B  2', '021 N30 E19 10Dbdc1', ...,
       '212 S19 E60 26Dcdc1', '176 N24 E58 16Bbcc1',
       '107 N11 E24 20Cbcb2'], dtype=object)

In [23]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['stream', 'Underground', 'Geothermal', 'Spring',
       'Other Surface Water', 'Other Ground Water', 'Effluent', 'lake',
       'Reservoir', 'Storage', 'Unknown'], dtype=object)

In [24]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['Denied', 'Certificate', 'Canceled',
       'Ready for Action (Protested)', 'Permit', 'Abrogated', 'Withdrawn',
       'Expired', 'Revocable Permit', 'Decreed', 'Relinquish a Portion',
       'Ready For Action', 'Vested Rights', 'Forfeited', 'Reserved',
       'Abandoned', 'Supersceded', 'Application', 'Rejected',
       'Relinquished', 'Revoked', 'Rescinded', 'Suspended', 'Curtailed'],
      dtype=object)

In [25]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['', 'Bright Holland Corporation A Nevada Corporation',
       'Sherlock Donald E Sherlock Arletta G', ..., 'Sloane Violet H',
       'Ehrhart Thomas A Jr',
       'Washburn Family Trust Dated February 19 2010'], dtype=object)

In [26]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array(['Irrigation', 'Municipal', 'Irrigation-DLE', 'Other', 'Commercial',
       'Quasi-Municipal', 'Mining and Milling', 'Industrial',
       'Stockwatering', 'Mining Milling and Dewatering', 'Domestic',
       'Wildlife', 'Environmental', 'As Decreed', 'Construction',
       'Dewatering', 'Recreational', 'Power', 'Storage',
       'Irrigation-Carey Act', 'Evaporation', 'Unknown'], dtype=object)

In [27]:
# in_Latitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna("")
outdf['in_Latitude'].unique()

array([41.988335  , 41.988679  , 41.93528   , ..., 38.8979158 ,
       41.30270767, 40.27783   ])

In [28]:
# in_Longitude
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna("")
outdf['in_Longitude'].unique()

array([-118.527229  , -118.636906  , -118.661667  , ..., -119.84807354,
       -117.98286335, -117.75976253])

In [29]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'])
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

array(['1954-09-23T00:00:00.000000000', '1959-05-14T00:00:00.000000000',
       '1970-12-07T00:00:00.000000000', ...,
       '2019-10-23T00:00:00.000000000', '2019-09-30T00:00:00.000000000',
       '2019-09-17T00:00:00.000000000'], dtype='datetime64[ns]')

In [30]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

array([''], dtype=object)

In [31]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

array([746.67, 32.58, 44.96, ..., 9.87, 948.952, 549.11], dtype=object)

In [32]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip()
        try:
            outList = WaterSourceNativeIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

array(['wadeID1', 'wadeID2', 'wadeID3', 'wadeID4', 'wadeID5', 'wadeID6'],
      dtype=object)

## Drop non-Active AllocationLegalStatusCV Water Rights
- For NV, we don't want water rights that are considered: Abandoned, Abrogated, Application, Canceled, Denied, Expired, Forfeited, Ready For Action, Ready for Action (Protested), Rejected, Revoked, Supersceded, Withdrawn

In [33]:
# drop non-active AllocationLegalStatusCV values specific to that state.

# drop the list
dropLegalStatusList = ["Abandoned", "Abrogated", "Application", "Canceled", "Denied", "Expired", "Forfeited", "Ready For Action", "Ready for Action (Protested)", "Rejected", "Revoked", "Supersceded", "Withdrawn"]

# drop rows from above list
outdf = outdf[outdf.in_AllocationLegalStatusCV.isin(dropLegalStatusList) == False].reset_index(drop=True)

print(len(outdf))
outdf['in_AllocationLegalStatusCV'].unique()

60833


array(['Certificate', 'Permit', 'Revocable Permit', 'Decreed',
       'Relinquish a Portion', 'Vested Rights', 'Reserved',
       'Relinquished', 'Rescinded', 'Suspended', 'Curtailed'],
      dtype=object)

## Shapefile Data
- For attaching gemetry to csv inputs.

In [34]:
# PoU Shapefile Data, Shapefile input
dfPoUshapetemp = gpd.read_file('shapefile/NVwr_POU.zip')
#dfPoUshapetemp = pd.DataFrame(dfPoUshapetemp)
dfPoUshapetemp.head(3)

,app,cert,app_status,mou,site_name,permit_inf,basin,county,source,duty_balan,div_balanc,prior_dt,pou_acre_t,poly_id,wadeSiteID,Latitude,Longitude,GlobalID,Shape_Leng,Shape_Area,geometry
0,V11569,None,VST,COM,036 N39 E52 15DBCC2,http://water.nv.gov/permitinformation.aspx?app...,036,EL,UG,0.00000,0.20000,1929-01-01,0.00000,69906,wadeID1,41.27617,-116.11752,{A4691461-1E67-43AD-9D4D-D525764C8A22},2309.90802,194139.97513,"MULTIPOLYGON (((-116.12022 41.27816, -116.1152..."
1,V11519,None,VST,IRR,033A N43 E37 34BBBD1,http://water.nv.gov/permitinformation.aspx?app...,033A,HU,UG,0.00000,0.00000,1923-01-01,0.00000,70212,wadeID2,41.56777,-117.79997,{5AA23025-1327-41BB-8083-FB1754B95C25},4004.29090,640308.96333,"POLYGON ((-117.80959 41.56965, -117.80478 41.5..."
2,V11518,None,VST,IRR,033A N43 E37 29DDCB1,http://water.nv.gov/permitinformation.aspx?app...,033A,HU,UG,0.00000,0.00000,1919-01-01,0.00000,70211,wadeID3,41.57357,-117.83477,{8CFB80D4-B00D-4A2F-9F1A-65A342B5141C},2851.62099,499295.62322,"POLYGON ((-117.83844 41.56996, -117.83852 41.5..."


In [35]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['wadeSiteID']
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

,in_SiteNativeID,geometry
0,POUwadeID1,"MULTIPOLYGON (((-116.12022 41.27816, -116.1152..."
1,POUwadeID2,"POLYGON ((-117.80959 41.56965, -117.80478 41.5..."
2,POUwadeID3,"POLYGON ((-117.83844 41.56996, -117.83852 41.5..."


### Exporting to Finished File

In [36]:
outdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60833 entries, 0 to 60832
Data columns (total 63 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   WaDEUUID                                      60833 non-null  object        
 1   in_MethodUUID                                 60833 non-null  object        
 2   in_VariableSpecificUUID                       60833 non-null  object        
 3   in_OrganizationUUID                           60833 non-null  object        
 4   in_Geometry                                   60833 non-null  object        
 5   in_GNISFeatureNameCV                          60833 non-null  object        
 6   in_WaterQualityIndicatorCV                    60833 non-null  object        
 7   in_WaterSourceName                            60833 non-null  object        
 8   in_WaterSourceNativeID                        60833 non-null  obje

In [37]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,nvD1,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID2,Groundwater,,Digitized,Humboldt,4326,,,,41.98868,-118.63691,,,POD,001 N47 E30 04B 2,POD1,,Underground,NV,,,,,,,,,,,Certificate,18001,,1959-05-14,,,,,32.58000,Irrigation,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
1,nvD100000,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID2,Groundwater,,Digitized,Lyon,4326,,,,39.33265,-119.43568,,,POD,103 N17 E23 18Ddda1,POD6,,Underground,NV,,,,,,,,,,,Permit,83807,Eitel Thomas H Sell Larry L And Rhonda L,1972-04-20,,,,,4.97000,Commercial,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
2,nvD100002,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID2,Groundwater,,Digitized,Lyon,4326,,,,39.33500,-119.42917,,,POD,103 N17 E23 17Cbbb2,POD8,,Underground,NV,,,,,,,,,,,Permit,83806,Eitel Thomas H Sell Larry R And Rhonda L,1972-04-20,,,,,4.97000,Commercial,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
3,nvD100003,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID2,Groundwater,,Digitized,Lyon,4326,,,,39.33265,-119.43568,,,POD,103 N17 E23 18Ddda1,POD9,,Underground,NV,,,,,,,,,,,Permit,83808,Eitel Thomas H Sell Larry L And Rhonda L,1972-04-10,,,,,3.15000,Commercial,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
4,nvD100004,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID2,Groundwater,,Digitized,Lyon,4326,,,,39.33265,-119.43568,,,POD,103 N17 E23 18Ddda1,POD10,,Underground,NV,,,,,,,,,,,Certificate,78703,Stagecoach Gid,1970-05-28,,,,,40.00000,Commercial,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60828,nvU29155,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID1,Surface Water,,Digitized,Esmerelda,4326,,,,37.79501,-117.68623,,,POU,143 S02 E39 22Aaca1,POUwadeID29156,,Spring,NV,,,,,,,,,,,Certificate,10036,,1936-10-10,,,,,,Mining and Milling,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
60829,nvU29156,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID1,Surface Water,,Digitized,Douglas,4326,,,,38.89792,-119.84807,,,POU,105 N12 E19 16Dcdb1,POUwadeID29157,,Spring,NV,,,,,,,,,,,Certificate,10033,,1936-10-07,,,,,36.20000,Irrigation,,,,,,0,,,,,,,,,,http://water.nv.gov/permitinformation.aspx?app...
60830,nvU29157,NVwr_M1,NVwr_V1,NVwr_O1,,,,,wadeID1,Surface Water,,Digitized,Humboldt,4326,,,,41.30271,-117.98286,,,POU,032 N40 E36 31Bcad1,POUwadeID29158,,Spring,NV,,,,,,,,,,,Certificate,10024,,1936-09-29,,,,,7.23900,Mining and Milling,,,,,,0,,,,,,,,,,http://water.nv.gov/perm

In [38]:
# Export the output dataframe
outdf.to_csv('Pwr_nvMain.zip', compression=dict(method='zip', archive_name='Pwr_nvMain.csv'), index=False)  # The output, save as a zip
dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.